In [5]:
import pandas as pd
import numpy as np
import torch
import random
data = pd.read_excel(r'model_data\东华\data_preprocessed.xlsx',  header=0)

In [91]:
ctrl_o2 = data["预热器前烟道左侧氧量2"]
ctrl_coal = data[["给煤机A瞬时煤量反馈信号", "给煤机B瞬时煤量反馈信号", "给煤机C瞬时煤量反馈信号", 
             "给煤机D瞬时煤量反馈信号", "给煤机E瞬时煤量反馈信号"]]
ctrl_sec = data[["2号角9层二次风门位置反馈", "2号角8层二次风门位置反馈", "2号角7层二次风门位置反馈", 
                 "2号角6层二次风门位置反馈", "2号角5层二次风门位置反馈", "2号角4层二次风门位置反馈", 
                 "2号角3层二次风门位置反馈", "2号角2层二次风门位置反馈", "2号角1层二次风门位置反馈"]]
ctrl_sofa = data[["2号角1层燃尽风门位置反馈", "2号角3层燃尽风门位置反馈", "2号角4层燃尽风门位置反馈", 
                  "2号角5层燃尽风门位置反馈"]]

### 绘图

In [ ]:
### 

In [58]:
state_all = data[["#1机组负荷", "主蒸汽温度（左）", "再热蒸汽压力左", 
                  "再热器减温水A侧流量", "A磨煤机入口一次风流量", "B磨煤机入口一次风流量",
                  "C磨煤机入口一次风流量", "D磨煤机入口一次风流量", "E磨煤机入口一次风流量"]]
target_etem = data["空预器A出口烟气温度4"]
target_rtem = data["再热蒸汽温度（左）"]
target_co = data["co_filter"]
target_nox = data["nox_filter"]

### 变量的描述信息

In [37]:
def print_distribution(data, points=[0.1,1,2,3,97,98,99,99.9]):
    for point in points:
        print(f"分位数{point}:",np.percentile(data, point))
    data_inc = (np.roll(data, -1) - data)[:-1]
    for point in points:
        print(f"增量分位数{point}:",np.percentile(data_inc, point))

print_distribution(np.array(target_nox), points=[0.1,99.9])    

分位数0.1: 131.0016920132
分位数99.9: 288.7898373884005
增量分位数0.1: -5.8711418235999915
增量分位数99.9: 6.723186430000121


In [84]:
ctrl_o2_limit = [[1.8], [4.8]]
ctrl_coal_limit = [[0, 0, 0, 0, 0], [40, 40, 40, 40, 40]]
ctrl_sec_limit = [[0, 0, 0, 0, 0, 0, 0, 0, 0], [75, 75, 75, 75, 75, 75, 75, 75, 75]]
ctrl_sofa_limit = [[0, 0, 0, 0], [100, 100, 100, 100]]
state_all_limit = [[110, 535, 1.1, 0, 0, 0, 0, 0, 0], [330, 550, 3.4, 2, 55, 55, 55, 55, 55]]
target_etem_limit = [[117], [157]]
target_co_limit = [[20], [1500]]
target_nox_limit = [[130], [280]]
target_rtem_limit = [[517], [547]]    

In [90]:
def shape2d(data):
    data = np.array(data)
    if(len(data.shape) == 1):
        return data.reshape(len(data), 1)
    else:
        return data
    

def minMaxNormalize(data, min_value, max_value):
    return (data - min_value)/(max_value - min_value)


data_ctrl = np.concatenate([shape2d(ctrl_o2), shape2d(ctrl_coal), shape2d(ctrl_sec), shape2d(ctrl_sofa)], 1)
data_state = shape2d(state_all)
data_target = np.concatenate([shape2d(target_etem), shape2d(target_rtem), shape2d(target_co), shape2d(target_nox)], 1)

data_ctrl_limit = np.concatenate([shape2d(ctrl_o2_limit), shape2d(ctrl_coal_limit), shape2d(ctrl_sec_limit), shape2d(ctrl_sofa_limit)], 1)
data_state_limit = shape2d(state_all_limit)
data_target_limit = np.concatenate([shape2d(target_etem_limit), shape2d(target_rtem_limit), shape2d(target_co_limit), shape2d(target_nox_limit)], 1)

assert data_ctrl.shape[1] == data_ctrl_limit.shape[1]
assert data_state.shape[1] == data_state_limit.shape[1]
assert data_target.shape[1] == data_target_limit.shape[1]

# normalize
data_ctrl = minMaxNormalize(data_ctrl, data_ctrl_limit[0, :], data_ctrl_limit[1, :])
data_state = minMaxNormalize(data_state, data_state_limit[0, :], data_state_limit[1, :])
data_target = minMaxNormalize(data_target, data_target_limit[0, :], data_target_limit[1, :])

### 建立多个预测模型